In [1]:
#! /usr/bin/env python
"""
Compute elevation statistics for the debris-covered areas in each latitude and longitude
"""

import sys
import os
import re
import subprocess
from datetime import datetime, timedelta
import time
import pickle
from collections import OrderedDict

import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import rasterio
from scipy import ndimage
import xarray as xr
from osgeo import gdal, ogr, osr

from pygeotools.lib import malib, warplib, geolib, iolib, timelib

import debrisglobal.globaldebris_input as debris_prms

In [2]:
# Debris cover extent shapefile with statistics
dc_shp = gpd.read_file(debris_prms.debriscover_fp + debris_prms.debriscover_fn_dict[debris_prms.roi])
dc_shp = dc_shp.sort_values(by=['RGIId'])

print('All DC glaciers:', dc_shp.shape[0], 'All DC Area (km2):', dc_shp.DC_Area_v2.sum() / 1e6)

# Subset by percent debris-covered or debris-covered area
dc_shp_subset = dc_shp[((dc_shp['DC_Area__1'] > debris_prms.dc_percarea_threshold) | 
                        (dc_shp['DC_Area_v2'] / 1e6 > debris_prms.dc_area_threshold))
                        & (dc_shp['Area'] > debris_prms.min_glac_area)].copy()
dc_shp_subset.reset_index(inplace=True, drop=True)

rgino_str_list_subset = [x.split('-')[1] for x in dc_shp_subset.RGIId.values]

print('Subset DC glaciers:', dc_shp_subset.shape[0], 'Subset DC Area (km2):', dc_shp_subset.DC_Area_v2.sum() / 1e6)

dc_shp_subset

All DC glaciers: 1011 All DC Area (km2): 174.176324
Subset DC glaciers: 43 Subset DC Area (km2): 92.158983


,RGIId,GLIMSId,BgnDate,EndDate,CenLon,CenLat,O1Region,O2Region,Area,Zmin,...,Name,DC_Area,DC_BgnDate,DC_EndDate,DC_CTSmean,DC_Area_%,area_singl,DC_Area_v2,DC_Area__1,geometry
0,RGI60-18.00171,G168039E44594S,19789999,-9999999,168.037603,-44.591427,18,1,2.790,1160,...,None,810000,2013,2017,20.105890,29.032,785599,785599,28.158,"POLYGON ((168.03242 -44.59846, 168.03280 -44.5..."
1,RGI60-18.00179,G168020E44606S,19789999,-9999999,168.018668,-44.605576,18,1,2.189,850,...,None,240300,2013,2017,24.129812,10.978,151562,151562,6.924,"POLYGON ((168.00657 -44.61615, 168.00694 -44.6..."
2,RGI60-18.00569,G168242E44591S,19789999,-9999999,168.235877,-44.590591,18,1,2.559,1503,...,None,373500,2013,2017,10.591865,14.596,133148,295086,11.531,"MULTIPOLYGON (((168.22521 -44.59034, 168.22597..."
3,RGI60-18.00631,G168624E44538S,19789999,-9999999,168.620338,-44.539415,18,1,2.472,1465,...,None,445500,2013,2017,11.149417,18.022,21598,343770,13.907,"MULTIPOLYGON (((168.61113 -44.54344, 168.61150..."
4,RGI60-18.00661,G168556E44463S,19789999,-9999999,168.553044,-44.465521,18,1,7.027,1073,...,None,1637100,2013,2017,11.708163,23.297,1614468,1614468,22.975,"POLYGON ((168.51256 -44.47246, 168.51332 -44.4..."
5,RGI60-18.00686,G168609E44455S,19789999,-9999999,168.602431,-44.456298,18,1,9.656,1036,...,None,1511100,2013,2017,11.280399,15.649,1165166,1429743,14.807,"MULTIPOLYGON (((168.60983 -44.45510, 168.61058..."
6,RGI60-18.00707,G168578E44439S,19789999,-9999999,168.578927,-44.439472,18,1,5.191,1282,...,None,741600,2013,2017,9.443480,14.286,25197,691794,13.327,"MULTIPOLYGON (((168.56455 -44.42875, 168.56530..."
7,RGI60-18.00716,G168482E44469S,19789999,-9999999,168.481966,-44.469631,18,1,4.053,1292,...,None,583200,2013,2017,11.178898,14.389,519199,540795,13.343,"MULTIPOLYGON (((168.47778 -44.46549, 168.47854..."
8,RGI60-18.00718,G168495E44485S,19789999,-9999999,168.487470,-44.487016,18,1,2.311,1136,...,None,387000,2013,2017,9.974349,16.746,158259,322029,13.935,"MULTIPOLYGON (((168.47985 -44.49632, 168.48023..."
9,RGI60-18.00815,G168358E44502S,19789999,-9999999,168.355187,-44.499472,18,1,2.833,1186,...,None,749700,2013,2017,10.124881,26.463,683812,683812,24.137,"POLYGON ((168.35942 -44.50602, 168.35980 -44.5..."


In [3]:
# Select glaciers using RGI and find unique latlons
#  (Scherler DC shapefiles do not have same CenLat and CenLon for some reason)
main_glac_rgi_subset = debris_prms.selectglaciersrgitable(rgino_str_list_subset)
main_glac_rgi_subset['CenLon_360'] = main_glac_rgi_subset['CenLon']
main_glac_rgi_subset.loc[main_glac_rgi_subset['CenLon_360'] < 0, 'CenLon_360'] = (
    360 + main_glac_rgi_subset.loc[main_glac_rgi_subset['CenLon_360'] < 0, 'CenLon_360'])

# Load met data and find nearest latlon indices
ds = xr.open_dataset(debris_prms.metdata_fp + '../' + debris_prms.metdata_elev_fn)
#  argmin() finds the minimum distance between the glacier lat/lon and the GCM pixel
lat_nearidx = (np.abs(main_glac_rgi_subset['CenLat'].values[:,np.newaxis] - 
                      ds['latitude'][:].values).argmin(axis=1))
lon_nearidx = (np.abs(main_glac_rgi_subset['CenLon_360'].values[:,np.newaxis] - 
                      ds['longitude'][:].values).argmin(axis=1))
latlon_nearidx = list(zip(lat_nearidx, lon_nearidx))
latlon_nearidx_unique = sorted(list(set(latlon_nearidx)))
main_glac_rgi_subset['latlon_nearidx'] = latlon_nearidx
latlon_unique_dict = dict(zip(latlon_nearidx_unique,np.arange(0,len(latlon_nearidx_unique))))
latlon_unique_dict_reversed = dict(zip(np.arange(0,len(latlon_nearidx_unique)),latlon_nearidx_unique))
main_glac_rgi_subset['latlon_unique_no'] = main_glac_rgi_subset['latlon_nearidx'].map(latlon_unique_dict)

print('unique lat/lons:', len(np.unique(main_glac_rgi_subset['latlon_unique_no'])), '\n\n')

# Delete me
latlon_nearidx_unique_v1 = latlon_nearidx_unique.copy()

lat_list = np.array([ds.latitude[x[0]].values for x in latlon_nearidx_unique])
lon_list = np.array([ds.longitude[x[1]].values for x in latlon_nearidx_unique])
latlon_list = list(tuple(zip(list(lat_list), list(lon_list))))

# Pickle unique lat/lons that will be used for melt model
with open(debris_prms.latlon_unique_fp + debris_prms.latlon_unique_dict[debris_prms.roi], 'wb') as f:
    pickle.dump(latlon_list, f)

43 glaciers in region 18 are included in this model run: ['00171', '00179', '00569', '00631', '00661', '00686', '00707', '00716', '00718', '00815', '00853', '00891', '00947', '00996', '01018', '01130', '01307', '01852', '01889', '01958', '01959', '02210', '02230', '02257', '02270', '02276', '02298', '02342', '02436', '02450', '02472', '02499', '02502', '02504', '02505', '02508', '03066', '03078', '03156', '03167', '03181', '03191', '03209'] and more
This study is focusing on 43 glaciers in region [18]
unique lat/lons: 12 




In [4]:
# ===== LOAD GLACIERS WITH DATA =====
main_glac_rgi_subset['mb_fn'] = np.nan
mb_binned_fp = debris_prms.mb_binned_fp

regions_str = [str(x).zfill(2) for x in debris_prms.roi_rgidict[debris_prms.roi]]

mb_fns = []
mb_rgiids = []
for i in os.listdir(mb_binned_fp):
    if i.endswith('_mb_bins.csv') and i.split('_')[0].split('.')[0].zfill(2) in regions_str:
        mb_fns.append(i)
        rgiid_raw = i.split('_')[0]
        rgiid = 'RGI60-' + rgiid_raw.split('.')[0].zfill(2) + '.' + rgiid_raw.split('.')[1]
        mb_rgiids.append(rgiid)
mb_rgiids = sorted(mb_rgiids)
mb_fns = sorted(mb_fns)
mb_fn_df = pd.DataFrame(np.zeros((len(mb_fns),2)), columns=['RGIId', 'mb_fn'])
mb_fn_df['RGIId'] = mb_rgiids
mb_fn_df['mb_fn'] = mb_fns

# Find glaciers that are debris-covered
mb_dc_rgiid = [value for value in list(mb_fn_df.RGIId.values) if value in list(main_glac_rgi_subset.RGIId.values)]
mb_fn_df_dc = mb_fn_df[mb_fn_df['RGIId'].isin(mb_dc_rgiid)]
mb_fn_df_dc = mb_fn_df_dc.sort_values('RGIId')

print('Debris-covered glaciers:', mb_fn_df_dc.shape[0], '\n\n')

mb_fn_dict = dict(zip(mb_fn_df_dc['RGIId'].values, mb_fn_df_dc['mb_fn'].values))

main_glac_rgi_subset['mb_fn'] = main_glac_rgi_subset.RGIId.map(mb_fn_dict)

Debris-covered glaciers: 43 




In [5]:
# ===== SELECT GLACIERS WITH DATA ====
main_glac_rgi_wobs = main_glac_rgi_subset.dropna(subset=['mb_fn']).copy()
# print('subset wdata length:', main_glac_rgi_wobs.shape)
main_glac_rgi_wobs.reset_index(inplace=True, drop=True)

# Update the latlon unique pickle files
latlon_nearidx_unique = sorted(list(set(main_glac_rgi_wobs['latlon_nearidx'].values)))
latlon_unique_dict = dict(zip(latlon_nearidx_unique,np.arange(0,len(latlon_nearidx_unique))))
latlon_unique_dict_reversed = dict(zip(np.arange(0,len(latlon_nearidx_unique)),latlon_nearidx_unique))
main_glac_rgi_wobs['latlon_unique_no'] = main_glac_rgi_wobs['latlon_nearidx'].map(latlon_unique_dict)

print('unique lat/lons:', len(np.unique(main_glac_rgi_wobs['latlon_unique_no'])), '\n\n')

lat_list = np.array([ds.latitude[x[0]].values for x in latlon_nearidx_unique])
lon_list = np.array([ds.longitude[x[1]].values for x in latlon_nearidx_unique])
latlon_list = list(tuple(zip(list(lat_list), list(lon_list))))

# Pickle unique lat/lons that will be used for melt model
with open(debris_prms.latlon_unique_fp + debris_prms.latlon_unique_dict[debris_prms.roi], 'wb') as f:
    pickle.dump(latlon_list, f)
    
main_glac_rgi_wobs

unique lat/lons: 12 




,O1Index,RGIId,CenLon,CenLat,O1Region,O2Region,Area,Zmin,Zmax,Zmed,...,TermType,Surging,RefDate,glacno,rgino_str,RGIId_float,CenLon_360,latlon_nearidx,latlon_unique_no,mb_fn
0,170,RGI60-18.00171,168.039,-44.5936,18,1,2.790,1160,2120,1407,...,0,9,19789999,171,18.00171,18.00171,168.039,"(538, 672)",8,18.00171_mb_bins.csv
1,178,RGI60-18.00179,168.020,-44.6058,18,1,2.189,850,2671,1806,...,0,9,19789999,179,18.00179,18.00179,168.020,"(538, 672)",8,18.00179_mb_bins.csv
2,568,RGI60-18.00569,168.242,-44.5907,18,1,2.559,1503,2172,1866,...,0,9,19789999,569,18.00569,18.00569,168.242,"(538, 673)",9,18.00569_mb_bins.csv
3,630,RGI60-18.00631,168.624,-44.5376,18,1,2.472,1465,2357,2050,...,0,9,19789999,631,18.00631,18.00631,168.624,"(538, 674)",10,18.00631_mb_bins.csv
4,660,RGI60-18.00661,168.556,-44.4630,18,1,7.027,1073,2553,1822,...,0,9,19789999,661,18.00661,18.00661,168.556,"(538, 674)",10,18.00661_mb_bins.csv
5,685,RGI60-18.00686,168.609,-44.4545,18,1,9.656,1036,2522,1832,...,0,9,19789999,686,18.00686,18.00686,168.609,"(538, 674)",10,18.00686_mb_bins.csv
6,706,RGI60-18.00707,168.578,-44.4393,18,1,5.191,1282,2514,1982,...,0,9,19789999,707,18.00707,18.00707,168.578,"(538, 674)",10,18.00707_mb_bins.csv
7,715,RGI60-18.00716,168.482,-44.4689,18,1,4.053,1292,2455,1985,...,0,9,19789999,716,18.00716,18.00716,168.482,"(538, 674)",10,18.00716_mb_bins.csv
8,717,RGI60-18.00718,168.495,-44.4852,18,1,2.311,1136,2432,1834,...,0,9,19789999,718,18.00718,18.00718,168.495,"(538, 674)",10,18.00718_mb_bins.csv
9,814,RGI60-18.00815,168.358,-44.5016,18,1,2.833,1186,2276,1597,...,0,9,19789999,815,18.00815,18.00815,168.358,"(538, 673)",9,18.00815_mb_bins.csv


In [6]:
# print('DELETE ME - HACK FOR DEVELOPMENT')
# print(np.where(main_glac_rgi_wobs['latlon_unique_no'] == 172)[0])
# main_glac_rgi_wobs = main_glac_rgi_wobs.loc[372:373,:]
# main_glac_rgi_wobs['mb_fn'].values

In [7]:
# ===== DEBRIS ELEVATION STATS ====================================================================================
# CALCULATE DEBRIS ELEVATION STATS FOR GLACIERS WITH DATA FOR EACH UNIQUE LAT/LON
elev_stats_latlon_dict = {}
latlon_list_updated = []
rgiid_4cal = []
for nlatlon, latlon_unique in enumerate(np.unique(main_glac_rgi_wobs.latlon_unique_no)):
# for nlatlon, latlon_unique in enumerate([np.unique(main_glac_rgi_wobs.latlon_unique_no)[0]]):

    main_glac_rgi_subset = main_glac_rgi_wobs[main_glac_rgi_wobs['latlon_unique_no'] == latlon_unique]
    main_glac_rgi_subset.reset_index(inplace=True, drop=True)
    
    # Debris elevation stats should be done by lat/lon
    df_all = None
    elev_list_all = []
    df_idx_count = 0
    count_width_passes = 0
    for nglac, glac_fn in enumerate(main_glac_rgi_subset.mb_fn.values):
#     for nglac, glac_fn in enumerate([main_glac_rgi_subset.mb_fn.values[0]]):

        glac_fullfn = debris_prms.mb_binned_fp + glac_fn
        
        glac_str_noleadzero = glac_fullfn.split('/')[-1].split('_')[0]
        rgiid = 'RGI60-' + glac_str_noleadzero.split('.')[0].zfill(2) + '.' + glac_str_noleadzero.split('.')[1]

        print(main_glac_rgi_subset.loc[nglac,'RGIId'], rgiid)
        
        # Select bins that meet calibratioin criteria
        df_raw = pd.read_csv(glac_fullfn)
        df = df_raw.dropna(subset=['mb_bin_mean_mwea'])
        df['z1_bin_areas_perc_cum'] = np.cumsum(df['z1_bin_area_valid_km2']) /df['z1_bin_area_valid_km2'].sum() * 100
        # add width to bins
        widths_fp = debris_prms.oggm_fp + 'widths/' + 'RGI60-' + rgiid.split('-')[1].split('.')[0] + '/'
        widths_fn = rgiid + '_widths_m.csv'
        try:
            # Add width to each elevation bin
            widths_df = pd.read_csv(widths_fp + widths_fn)
            elev_nearidx = (np.abs(df['bin_center_elev_m'].values[:,np.newaxis] - 
                                   widths_df['elev'].values).argmin(axis=1))
            df['width_m'] = widths_df.loc[elev_nearidx,'width_m'].values
        except:
            df['width_m'] = 0
        
        df_idx = np.where((df['vm_med'] <= debris_prms.vel_threshold) 
                          & (df['width_m'] >= debris_prms.width_min_dict[debris_prms.roi])
                          & (df['dc_bin_area_perc'] >= debris_prms.debrisperc_threshold)
                          & (df['dc_bin_count_valid'] >= 10)
                          & (df['z1_bin_areas_perc_cum'] <= debris_prms.term_area_perc)
                          )[0]
        df_debris = df.loc[df_idx,:]
        df_debris.reset_index(inplace=True, drop=True)
        df_idx_count += len(df_idx)
        
            
        if len(df_idx) > 0:
            for nelev, elev in enumerate(list(df_debris['bin_center_elev_m'].values)):
                elev_list_single = list(np.repeat(elev, df_debris.loc[nelev,'dc_bin_count_valid']))
                elev_list_all.extend(elev_list_single)
            
#             # only work with terminus
#             df_idx_dif = list(df_idx[1:] - df_idx[:-1])
#             if np.sum(df_idx_dif) == len(df_idx)-1:
#                 df_idx_nojump = df_idx
#             else:
#                 idx_jumpinbins = df_idx_dif.index(next(filter(lambda x: x>1, df_idx_dif)))
#                 df_idx_nojump = df_idx[0:idx_jumpinbins+1]
#             df_debris_nojump = df_debris.loc[df_idx_nojump,:]
#             df_debris_nojump.reset_index(inplace=True, drop=True)
#             # Median width to ensure terminus velocities can be estimated
#             width_median = np.median(widths_m[np.where(h < df_debris_nojump['bin_center_elev_m'].max())[0]])
#             if width_median > debris_prms.width_min_dict[debris_prms.roi]:
#                 for nelev, elev in enumerate(list(df_debris_nojump['bin_center_elev_m'].values)):
#                     elev_list_single = list(np.repeat(elev, df_debris_nojump.loc[nelev,'dc_bin_count_valid']))
#                     elev_list_all.extend(elev_list_single)
#                 count_width_passes += 1
    
            rgiid_4cal.append(rgiid.split('-')[1])
        
    if df_idx_count > 0:
        dc_zmean = np.mean(elev_list_all)
        dc_zstd = np.std(elev_list_all)
        dc_zmed = malib.fast_median(elev_list_all)
        dc_zmad = malib.mad(elev_list_all)
        
        lat_deg = float(ds.latitude[latlon_unique_dict_reversed[latlon_unique][0]].values)
        lon_deg = float(ds.longitude[latlon_unique_dict_reversed[latlon_unique][1]].values)
        elev_stats_latlon_dict[lat_deg,lon_deg] = [dc_zmean, dc_zstd, dc_zmed, dc_zmad]
        latlon_list_updated.append((lat_deg, lon_deg))
        
print('unique lat/lons updated:', len(latlon_list_updated))
# Update pickle of unique lat/lons that will be used for melt model
with open(debris_prms.latlon_unique_fp + debris_prms.latlon_unique_dict[debris_prms.roi], 'wb') as f:
    pickle.dump(latlon_list_updated, f)

RGI60-18.03066 RGI60-18.03066
RGI60-18.03078 RGI60-18.03078
RGI60-18.03167 RGI60-18.03167
RGI60-18.03181 RGI60-18.03181
RGI60-18.03156 RGI60-18.03156
RGI60-18.03191 RGI60-18.03191
RGI60-18.03209 RGI60-18.03209
RGI60-18.02230 RGI60-18.02230
RGI60-18.02257 RGI60-18.02257
RGI60-18.02436 RGI60-18.02436
RGI60-18.02502 RGI60-18.02502
RGI60-18.02210 RGI60-18.02210
RGI60-18.02270 RGI60-18.02270
RGI60-18.02276 RGI60-18.02276
RGI60-18.02342 RGI60-18.02342
RGI60-18.02450 RGI60-18.02450
RGI60-18.02472 RGI60-18.02472
RGI60-18.02298 RGI60-18.02298
RGI60-18.02499 RGI60-18.02499
RGI60-18.02504 RGI60-18.02504
RGI60-18.02505 RGI60-18.02505
RGI60-18.02508 RGI60-18.02508
RGI60-18.01852 RGI60-18.01852
RGI60-18.01889 RGI60-18.01889
RGI60-18.01958 RGI60-18.01958
RGI60-18.01959 RGI60-18.01959
RGI60-18.01130 RGI60-18.01130
RGI60-18.00947 RGI60-18.00947
RGI60-18.00996 RGI60-18.00996
RGI60-18.01307 RGI60-18.01307
RGI60-18.00171 RGI60-18.00171
RGI60-18.00179 RGI60-18.00179
RGI60-18.00569 RGI60-18.00569
RGI60-18.0

In [8]:
# Statistics of data coverage
rgiid_4cal = sorted(rgiid_4cal)
main_glac_rgi_4cal = debris_prms.selectglaciersrgitable(rgiid_4cal)
dc_area_dict = dict(zip(dc_shp.RGIId.values, dc_shp.DC_Area_v2.values))
main_glac_rgi_4cal['DC_Area_v2'] = main_glac_rgi_4cal.RGIId.map(dc_area_dict)
print('\nDC glaciers (used for cal):', main_glac_rgi_4cal.shape[0], 
      'DC Area (used for cal, km2):', main_glac_rgi_4cal.DC_Area_v2.sum() / 1e6)

39 glaciers in region 18 are included in this model run: ['00179', '00631', '00661', '00686', '00707', '00716', '00718', '00815', '00853', '00891', '00947', '00996', '01018', '01130', '01307', '01852', '01889', '01958', '01959', '02210', '02230', '02270', '02276', '02298', '02342', '02436', '02472', '02499', '02502', '02504', '02505', '02508', '03066', '03078', '03156', '03167', '03181', '03191', '03209'] and more
This study is focusing on 39 glaciers in region [18]

DC glaciers (used for cal): 39 DC Area (used for cal, km2): 89.020948


In [9]:
# ===== ADD DEBRIS ELEVATION STATS TO MET DATA ======
overwrite_dc_stats = True
for nlatlon, latlon in enumerate(latlon_list_updated):
# for nlatlon, latlon in enumerate([latlon_list_updated[0]]):
    
    lat_deg = latlon[0]
    lon_deg = latlon[1]
    
    print(nlatlon, lat_deg, lon_deg)
    
    if lat_deg < 0:
        lat_str = 'S-'
    else:
        lat_str = 'N-' 

    # ===== Meteorological data =====
    metdata_fn = debris_prms.metdata_fn_sample.replace(
        'XXXX', str(int(np.abs(lat_deg)*100)) + lat_str + str(int(lon_deg*100)) + 'E-')
    
    ds = xr.open_dataset(debris_prms.metdata_fp + metdata_fn) 
    try:
        print('  existed:', ds.dc_zmean.values, 'vs', elev_stats_latlon_dict[latlon][0])
    except:
        pass
    if 'dc_zmean' not in list(ds.keys()) or overwrite_dc_stats:
        # Add stats
        ds['dc_zmean'] = elev_stats_latlon_dict[latlon][0]
        ds['dc_zmean'].attrs = {'units':'m a.s.l.', 'long_name':'Mean debris cover elevation', 
                                'comment':'converted from debris cover with data that will be used for subdebris melt inversion'}
        ds['dc_zstd'] = elev_stats_latlon_dict[latlon][1]
        ds['dc_zstd'].attrs = {'units':'m a.s.l.', 'long_name':'Standard deviation of debris cover elevation', 
                               'comment':'converted from debris cover with data that will be used for subdebris melt inversion'}
        ds['dc_zmed'] = elev_stats_latlon_dict[latlon][2]
        ds['dc_zmed'].attrs = {'units':'m a.s.l.', 'long_name':'Median debris cover elevation', 
                               'comment':'converted from debris cover with data that will be used for subdebris melt inversion'}
        ds['dc_zmad'] = elev_stats_latlon_dict[latlon][3]
        ds['dc_zmad'].attrs = {'units':'m a.s.l.', 'long_name':'Median absolute deviation of debris cover elevation', 
                               'comment':'converted from debris cover with data that will be used for subdebris melt inversion'}

        try:
            ds.close()
        except:
            continue
            
        # Export updated dataset
        ds.to_netcdf(debris_prms.metdata_fp + metdata_fn, mode='a')
    else:
        print(lat_deg, lon_deg, 'exists')

0 -43.25 170.75
  existed: 1231.7614678899083 vs 1233.2008567348882
1 -43.25 171.0
  existed: 1180.7060755336618 vs 1180.7060755336618
2 -43.5 170.0
  existed: 827.355871886121 vs 827.355871886121
3 -43.5 170.25
  existed: 1036.6117216117216 vs 1041.164316966456
4 -43.5 170.5
  existed: 1121.8842945230324 vs 1122.3825991991264
5 -43.75 170.0
  existed: 1069.362657091562 vs 1069.362657091562
6 -44.0 169.5
  existed: 1670.0393700787401 vs 1670.0393700787401
7 -44.25 168.75
  existed: 1253.0388978930307 vs 1253.0388978930307
8 -44.5 168.0
  existed: 942.3633440514469 vs 942.3633440514469
9 -44.5 168.25
  existed: 1028.584 vs 1026.4147130153597
10 -44.5 168.5
  existed: 1289.0159189580318 vs 1297.0892575039495
11 -44.5 168.75
  existed: 1389.0 vs 1389.0


In [10]:
print('DONE!')

DONE!


In [11]:
# ==== OLD FILE OF LOADIING MULTIPLE DATASETS =====
# # ===== LOAD GLACIERS WITH LARSEN DATA =====
# dc_shp_subset['larsen_fullfn'] = np.nan
# larsen_fullfn_dict = {}
# if 'larsen' in input.mb_datasets:
#     mb_summary = pd.read_csv(input.larsen_fp + input.larsen_fn)
    
#     # Find glaciers that are debris-covered
#     larsen_dc_rgiid = [value for value in list(mb_summary.RGIId.values) 
#                        if value in list(dc_shp_subset.RGIId.values)]

#     mb_summary_dc = mb_summary[mb_summary['RGIId'].isin(larsen_dc_rgiid)]
#     mb_summary_dc = mb_summary_dc.sort_values('RGIId')
#     mb_summary_dc.reset_index(inplace=True, drop=True)
#     mb_summary_dc.loc[mb_summary_dc['name'] == 'Maclaren', 'name'] = 'MacLaren'
#     mb_summary_dc.loc[mb_summary_dc['name'] == 'Tlikakila Fork', 'name'] = 'TlikakilaGlacierFork'
#     mb_summary_dc.loc[mb_summary_dc['name'] == 'Tlikakila N. Fork', 'name'] = 'TlikakilaNorthFork'
#     mb_summary_dc['larsen_fullfn'] = np.nan
    
#     for n, glac_name in enumerate(mb_summary_dc.name.values):
# #     for n, glac_name in enumerate([mb_summary_dc.name.values[47]]):
# #         print(n, glac_name)
            
#         glac_name = glac_name.replace(' ', '')
#         glac_fns = []
#         start_yr = []
#         end_yr = []
#         for i in os.listdir(input.larsen_binned_fp):
#             if i.startswith(glac_name):
#                 glac_fns.append(i)
#                 start_yr.append(i.split('.')[1][0:4])
#                 end_yr.append(i.split('.')[2][0:4])
                
#         if len(glac_fns) > 0:
#             yr_dif = np.array(end_yr).astype(int) - np.array(start_yr).astype(int)
#             mb_fn = glac_fns[np.where(yr_dif == yr_dif.max())[0][0]]
            
#             # ===== Process Larsen dataset =====
#             larsen_data_raw = np.genfromtxt(input.larsen_binned_fp + mb_fn, skip_header=3)
#             larsen_data_header = ['E', 'DZ', 'DZ25', 'DZ75', 'AAD', 'MassChange', 'MassBal', 'NumData']
#             larsen_data = pd.DataFrame(larsen_data_raw, columns=larsen_data_header)
#             larsen_data['std from DZ25'] = np.absolute(larsen_data['DZ'] - larsen_data['DZ25']) / 0.67
#             larsen_data['std from DZ75'] = np.absolute(larsen_data['DZ'] - larsen_data['DZ75']) / 0.67
#             larsen_data[' dhdt_bin_std_ma'] = (larsen_data['std from DZ25'] + larsen_data['std from DZ75']) / 2
#             larsen_data[' mb_bin_std_mwea'] = larsen_data[' dhdt_bin_std_ma'] * 900 / 1000
#             larsen_data['AAD'] = larsen_data['AAD'] / 1e6
#             larsen_data['startyear'] = int(mb_fn.split('.')[1][0:4])
#             larsen_data['endyear'] = int(mb_fn.split('.')[2][0:4])
#             larsen_data = larsen_data.rename({'E': '# bin_center_elev_m',
#                                               'DZ': ' dhdt_bin_mean_ma',
#                                               'MassBal': ' mb_bin_mean_mwea',
#                                               'AAD': ' z1_bin_area_valid_km2',
#                                              }, axis='columns')
#             new_fn = mb_summary_dc.loc[n,'RGIId'].split('-')[1][1:] + '_larsen_mb_bins.csv'
#             larsen_data.to_csv(input.larsen_binned_fp + new_fn, index=False)
            
#             mb_summary_dc.loc[n, 'larsen_fullfn'] = input.larsen_binned_fp + new_fn
            
#         else:
#             print(n, glac_name, 'has no file\n')

#     mb_summary_dc.dropna(subset=['larsen_fullfn'], inplace=True)
#     mb_summary_dc.reset_index(inplace=True, drop=True)
    
#     print('Larsen debris-covered glaciers:', mb_summary_dc.shape[0], '\n\n')
    
#     larsen_fullfn_dict = dict(zip(mb_summary_dc['RGIId'].values, mb_summary_dc['larsen_fullfn'].values))
# #     print(larsen_fullfn_dict)
#     dc_shp_subset['larsen_fullfn'] = dc_shp_subset.RGIId.map(larsen_fullfn_dict)

# # ===== LOAD GLACIERS WITH BRAUN DATA =====
# dc_shp_subset['braun_fullfn'] = np.nan
# braun_fullfn_dict = {}
# if 'braun' in input.mb_datasets:
#     mb_binned_fp = input.main_directory + '/../mb_data/Braun/binned_data/'
# #     mb_binned_fp = input.mb_binned_fp
    
#     mb_fns = []
#     braun_rgiids = []
#     for i in os.listdir(mb_binned_fp):
#         if i.endswith('_mb_bins.csv'):
#             mb_fns.append(mb_binned_fp + i)
#             rgiid_raw = i.split('_')[0]
#             rgiid = 'RGI60-' + rgiid_raw.split('.')[0].zfill(2) + '.' + rgiid_raw.split('.')[1]
#             braun_rgiids.append(rgiid)
#     braun_fn_df = pd.DataFrame(np.zeros((len(mb_fns),2)), columns=['RGIId', 'braun_fn'])
#     braun_fn_df['RGIId'] = braun_rgiids
#     braun_fn_df['braun_fullfn'] = mb_fns
    
#     # Find glaciers that are debris-covered
#     braun_dc_rgiid = [value for value in list(braun_fn_df.RGIId.values) 
#                        if value in list(dc_shp_subset.RGIId.values)]
#     braun_fn_df_dc = braun_fn_df[braun_fn_df['RGIId'].isin(braun_dc_rgiid)]
#     braun_fn_df_dc = braun_fn_df_dc.sort_values('RGIId')
    
#     print('Braun debris-covered glaciers:', braun_fn_df_dc.shape[0], '\n\n')
    
#     braun_fullfn_dict = dict(zip(braun_fn_df_dc['RGIId'].values, braun_fn_df_dc['braun_fullfn'].values))
    
#     dc_shp_subset['braun_fullfn'] = dc_shp_subset.RGIId.map(braun_fullfn_dict)

# # ===== LOAD GLACIERS WITH SHEAN DATA =====
# dc_shp_subset['shean_fullfn'] = np.nan
# shean_fullfn_dict = {}
# if 'shean' in input.mb_datasets:
# #     mb_binned_fp = input.main_directory + '/../mb_data/Shean_2019_0213/mb_combined_20190213_nmad_bins/'
#     mb_binned_fp = input.mb_binned_fp
    
#     mb_fns = []
#     rgiids = []
#     for i in os.listdir(mb_binned_fp):
#         if i.endswith('_mb_bins.csv'):
#             mb_fns.append(mb_binned_fp + i)
#             rgiid_raw = i.split('_')[0]
#             rgiid = 'RGI60-' + rgiid_raw.split('.')[0].zfill(2) + '.' + rgiid_raw.split('.')[1]
#             rgiids.append(rgiid)
#     mb_fn_df = pd.DataFrame(np.zeros((len(mb_fns),2)), columns=['RGIId', 'mb_fn'])
#     mb_fn_df['RGIId'] = rgiids
#     mb_fn_df['mb_fullfn'] = mb_fns
    
#     # Find glaciers that are debris-covered
#     mb_dc_rgiid = [value for value in list(mb_fn_df.RGIId.values) 
#                    if value in list(dc_shp_subset.RGIId.values)]
#     mb_fn_df_dc = mb_fn_df[mb_fn_df['RGIId'].isin(mb_dc_rgiid)]
#     mb_fn_df_dc = mb_fn_df_dc.sort_values('RGIId')
    
#     print('shean debris-covered glaciers:', mb_fn_df_dc.shape[0], '\n\n')
    
#     shean_fullfn_dict = dict(zip(mb_fn_df_dc['RGIId'].values, mb_fn_df_dc['mb_fullfn'].values))
# #     print(shea_fullfn_dict)
#     dc_shp_subset['shean_fullfn'] = dc_shp_subset.RGIId.map(shean_fullfn_dict)

# # Merge dictionaries together
# mb_fn_dict = dict(list(larsen_fullfn_dict.items()) + list(braun_fullfn_dict.items()) + 
#                   list(shean_fullfn_dict.items()))